In [0]:
import requests
import pandas as pd

# Your paths
catalog = "workspace"
schema  = "crime"
volume = "datastore"
download_url = "https://data.lacity.org/resource/2nrs-mtv8.csv?$limit=10000000"  # Added limit parameter
file_name = "crime_data.csv"
table_name = "crime_bronze"
path_volume = "/Volumes/" + catalog + "/" + schema + "/" + volume
path_table = catalog + "." + schema 

print(f"Volume path: {path_volume}")
print(f"Table path: {path_table}")

# Full file path in the volume
file_path = f"{path_volume}/{file_name}"
print(f"File will be saved to: {file_path}")

# Step 1: Download the data with pandas (handles query params better)
print("Downloading data from LA City portal...")
df_pandas = pd.read_csv(download_url)
print(f"Downloaded {len(df_pandas):,} rows")

# Step 2: Save to Unity Catalog volume
print(f"Saving to {file_path}...")
df_pandas.to_csv(f"/Volumes/{catalog}/{schema}/{volume}/{file_name}", index=False)
print("File saved successfully!")

# Step 3: Read into Spark DataFrame
print("Reading into Spark DataFrame...")
df_spark = spark.read.csv(
    file_path,
    header=True,
    inferSchema=True
)

print(f"\n✅ Success!")
print(f"   Rows: {df_spark.count():,}")
print(f"   File: {file_path}")

# Display sample
display(df_spark)

Data Profiling

In [0]:
# Load the bronze table
catalog = "workspace"
schema = "crime"
table_name = "crime_bronze"

df = spark.table(f"{catalog}.{schema}.{table_name}")

print("="*80)
print("STEP 1: BASIC DATA INFORMATION")
print("="*80)
print(f"\nTotal Rows: {df.count():,}")
print(f"Total Columns: {len(df.columns)}")
print(f"\nColumn Names:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i}. {col}")

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd

# Configuration
catalog = "workspace"
schema = "crime"
volume = "datastore"
table_name = "crime_bronze"

# Load bronze data
df = spark.table(f"{catalog}.{schema}.{table_name}")

print("="*80)
print("STEP 1: DATA PROFILING")
print("="*80)

# Basic statistics
total_rows = df.count()
total_cols = len(df.columns)

print(f"\n1.1 DATASET OVERVIEW")
print(f"{'='*80}")
print(f"Total Records: {total_rows:,}")
print(f"Total Columns: {total_cols}")

# Check duplicates
duplicate_cases = df.groupBy('DR_NO').count().filter(col('count') > 1).count()
print(f"Duplicate Case Numbers: {duplicate_cases:,}")

# Display schema
print(f"\n1.2 DATA TYPES")
print(f"{'='*80}")
df.printSchema()

In [0]:
print(f"\n1.3 NULL VALUE ANALYSIS")
print(f"{'='*80}")
# Calculate nulls for all columns
null_analysis = []
for column_name in df.columns:
    null_count = df.filter(col(column_name).isNull()).count()
    null_pct = (null_count / total_rows) * 100
    null_analysis.append({
        'Column': column_name,
        'Null_Count': null_count,
        'Null_Percentage': float(f"{null_pct:.2f}")  # Format instead of round
    })
null_df = pd.DataFrame(null_analysis).sort_values('Null_Percentage', ascending=False)
print(null_df.to_string(index=False))

This is a data quality audit to identify problems in the date fields before analysis.

In [0]:
print(f"\n1.5 DATE QUALITY ISSUES")
print(f"{'='*80}")

# Convert dates
df_dates = df.withColumn('date_reported', to_date(col('date_rptd'))) \
             .withColumn('date_occurred', to_date(col('date_occ')))

# Future dates
future_reported = df_dates.filter(col('date_reported') > current_date()).count()
future_occurred = df_dates.filter(col('date_occurred') > current_date()).count()

# Illogical dates (occurrence after report)
occ_after_report = df_dates.filter(col('date_occurred') > col('date_reported')).count()

# Null dates
null_reported = df.filter(col('date_rptd').isNull()).count()
null_occurred = df.filter(col('date_occ').isNull()).count()

print(f"Date Issues:")
print(f"  Null Report Dates:           {null_reported:,}")
print(f"  Null Occurrence Dates:       {null_occurred:,}")
print(f"  Future Report Dates:         {future_reported:,}")
print(f"  Future Occurrence Dates:     {future_occurred:,}")
print(f"  Occurrence > Report Date:    {occ_after_report:,}")

# Date ranges
date_stats = df_dates.select(
    min('date_occurred').alias('min_date'),
    max('date_occurred').alias('max_date')
).collect()[0]
print(f"\nDate Range: {date_stats['min_date']} to {date_stats['max_date']}")

This helps identify data entry errors where times were recorded incorrectly!

In [0]:
print(f"\n1.6 TIME QUALITY ISSUES")
print(f"{'='*80}")

# Time statistics
null_time = df.filter(col('time_occ').isNull()).count()
invalid_time = df.filter((col('time_occ') < 0) | (col('time_occ') > 2359)).count()

print(f"Time Issues:")
print(f"  Null Times:                  {null_time:,}")
print(f"  Invalid Times (<0 or >2359): {invalid_time:,}")

# Time range
time_stats = df.select(min('time_occ'), max('time_occ')).collect()[0]
print(f"\nTime Range: {time_stats[0]} to {time_stats[1]}")

In [0]:
print(f"\n1.7 COORDINATE QUALITY ISSUES")
print(f"{'='*80}")

# Coordinate issues
null_lat = df.filter(col('lat').isNull()).count()
null_lon = df.filter(col('lon').isNull()).count()
zero_lat = df.filter(col('lat') == 0).count()
zero_lon = df.filter(col('lon') == 0).count()
both_zero = df.filter((col('lat') == 0) & (col('lon') == 0)).count()
both_null = df.filter(col('lat').isNull() & col('lon').isNull()).count()

# Valid coordinates
valid_coords = df.filter(
    (col('lat').isNotNull()) & (col('lon').isNotNull()) &
    (col('lat') != 0) & (col('lon') != 0)
).count()

# LA bounds check (Lat: 33.7-34.8, Lon: -118.7 to -118.0)
outside_la = df.filter(
    (col('lat').isNotNull()) & (col('lon').isNotNull()) &
    (col('lat') != 0) & (col('lon') != 0) &
    ((col('lat') < 33.7) | (col('lat') > 34.8) |
     (col('lon') < -118.7) | (col('lon') > -118.0))
).count()

print(f"Coordinate Issues:")
print(f"  Null Latitude:         {null_lat:,} ({null_lat/total_rows*100:.2f}%)")
print(f"  Null Longitude:        {null_lon:,} ({null_lon/total_rows*100:.2f}%)")
print(f"  Zero Latitude:         {zero_lat:,} ({zero_lat/total_rows*100:.2f}%)")
print(f"  Zero Longitude:        {zero_lon:,} ({zero_lon/total_rows*100:.2f}%)")
print(f"  Both Null:             {both_null:,} ({both_null/total_rows*100:.2f}%)")
print(f"  Both Zero:             {both_zero:,} ({both_zero/total_rows*100:.2f}%)")
print(f"  Outside LA Bounds:     {outside_la:,} ({outside_la/total_rows*100:.2f}%)")
print(f"  Valid Coordinates:     {valid_coords:,} ({valid_coords/total_rows*100:.2f}%)")

In [0]:
print(f"\n1.8 AGE QUALITY ISSUES")
print(f"{'='*80}")

# Age issues
null_age = df.filter(col('vict_age').isNull()).count()
negative_age = df.filter(col('vict_age') < 0).count()
zero_age = df.filter(col('vict_age') == 0).count()
over_120 = df.filter(col('vict_age') > 120).count()
valid_age = df.filter((col('vict_age') >= 1) & (col('vict_age') <= 120)).count()

print(f"Age Issues:")
print(f"  Null Ages:             {null_age:,} ({null_age/total_rows*100:.2f}%)")
print(f"  Negative Ages:         {negative_age:,} ({negative_age/total_rows*100:.2f}%)")
print(f"  Zero Ages:             {zero_age:,} ({zero_age/total_rows*100:.2f}%)")
print(f"  Ages > 120:            {over_120:,} ({over_120/total_rows*100:.2f}%)")
print(f"  Valid Ages (1-120):    {valid_age:,} ({valid_age/total_rows*100:.2f}%)")

# Age statistics
age_stats = df.filter((col('vict_age') >= 0) & (col('vict_age') <= 120)).select(
    min('vict_age').alias('min'),
    max('vict_age').alias('max'),
    avg('vict_age').alias('mean')
).collect()[0]

print(f"\nValid Age Statistics:")
print(f"  Min:  {age_stats['min']}")
print(f"  Max:  {age_stats['max']}")
print(f"  Mean: {age_stats['mean']:.1f}")

In [0]:
%sql
select * from workspace.crime.crime_bronze where vict_age < 10;